In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

C:\Users\ojare\anaconda3\envs\tensorflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Create your input pipeline

In [6]:
# load dataset
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [7]:
# build a trainable pipeline
def normalize_img(image, label):
    """Normalizes images: 'uint8' -> 'float32.'"""
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [8]:
# building an evaluation pipeline
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

#### Train the model

In [11]:
model = Sequential([
    layers.Flatten(input_shape=(28,28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test
)

Epoch 1/10
469/469 [==============================] - 5s 8ms/step - loss: 0.3538 - sparse_categorical_accuracy: 0.9016 - val_loss: 0.1928 - val_sparse_categorical_accuracy: 0.9447
Epoch 2/10
469/469 [==============================] - 4s 7ms/step - loss: 0.1637 - sparse_categorical_accuracy: 0.9534 - val_loss: 0.1318 - val_sparse_categorical_accuracy: 0.9631
Epoch 3/10
469/469 [==============================] - 4s 7ms/step - loss: 0.1160 - sparse_categorical_accuracy: 0.9668 - val_loss: 0.1091 - val_sparse_categorical_accuracy: 0.9675
Epoch 4/10
469/469 [==============================] - 4s 7ms/step - loss: 0.0888 - sparse_categorical_accuracy: 0.9745 - val_loss: 0.0911 - val_sparse_categorical_accuracy: 0.9721
Epoch 5/10
469/469 [==============================] - 4s 7ms/step - loss: 0.0719 - sparse_categorical_accuracy: 0.9789 - val_loss: 0.0906 - val_sparse_categorical_accuracy: 0.9734
Epoch 6/10
469/469 [==============================] - 4s 7ms/step - loss: 0.0588 - sparse_categorica